# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Change directory to where this file is located
"""
%cd '/content/drive/MyDrive/MLDL_lab/'

/content/drive/MyDrive/MLDL_lab


# Import Modules

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

#Utils

In [ ]:
def tanh(z):
    """
    Implement the tanh activation function.
    The method takes the input z and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####   
    output = (2 / (1 + np.exp(-2*z))) - 1
    return output
    #####################


def softmax(X):
    """
    Implement the softmax function.
    The method takes the input X and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####
    logit = np.exp(X - np.amax(X, axis = 1, keepdims = True))
    numer = logit
    denom = np.sum(logit, axis=1, keepdims=True)
    return numer/denom
    #####################


def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

#2-Layer Neural Network

In [ ]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.

        Question (b)
        - refer to https://paperswithcode.com/method/xavier-initialization for Xavier initialization 
        
        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        ##### YOUR CODE #####
        params["W1"] = np.random.uniform(-1 / np.sqrt(input_dim), 1 / np.sqrt(input_dim), (input_dim, num_hiddens))
        params["b1"] = np.zeros((1, num_hiddens))
        params["W2"] = np.random.uniform(-1 / np.sqrt(num_hiddens), 1 / np.sqrt(num_hiddens), (num_hiddens, num_classes))
        params["b2"] = np.zeros((1, num_classes))
        #####################
        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(tanh(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (c)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ff_dict = {}
        ##### YOUR CODE #####
        params = self.params

        ff_dict['s1'] = np.matmul(X, params['W1'])
        ff_dict['s2'] = ff_dict['s1'] + params['b1']
        ff_dict['s3'] = tanh(ff_dict['s2'])
        ff_dict['s4'] = np.matmul(ff_dict['s3'], params['W2'])
        ff_dict['s5'] = ff_dict['s4']  + params['b2']
        y = softmax(ff_dict['s5'])
        #####################
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (d)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        ##### YOUR CODE #####
        dl_ds = self.forward(X)[0] - Y

        ds_1 = np.matmul(dl_ds, self.params['W2'].T) * (1 - np.power(ff_dict['s3'], 2))
        grads["dW1"] = np.matmul(X.T, ds_1)
        grads["db1"] = np.mean(ds_1, axis=0, keepdims=True)
        grads["dW2"] = np.matmul(ff_dict['s3'].T, dl_ds)
        grads["db2"] = np.mean(dl_ds, axis=0, keepdims=True)
        #####################
        return grads


    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.
        
        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """        
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

#Load MNIST

In [ ]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


#Training & Evaluation

In [ ]:
### 
# Question (e)
# Tune the hyperparameters with validation data, 
# and print the results by running the lines below.
###

In [ ]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=128, num_classes=10)

In [ ]:
# train the model
lr, n_epochs, batch_size = 1.0, 15, 64
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.378 0.906, valid loss/acc: 0.412 0.902
epoch 01 - train loss/acc: 0.178 0.950, valid loss/acc: 0.244 0.942
epoch 02 - train loss/acc: 0.123 0.963, valid loss/acc: 0.193 0.951
epoch 03 - train loss/acc: 0.145 0.954, valid loss/acc: 0.217 0.941
epoch 04 - train loss/acc: 0.089 0.972, valid loss/acc: 0.176 0.958
epoch 05 - train loss/acc: 0.081 0.974, valid loss/acc: 0.172 0.957
epoch 06 - train loss/acc: 0.070 0.978, valid loss/acc: 0.164 0.961
epoch 07 - train loss/acc: 0.077 0.974, valid loss/acc: 0.180 0.956
epoch 08 - train loss/acc: 0.034 0.990, valid loss/acc: 0.144 0.967
epoch 09 - train loss/acc: 0.047 0.985, valid loss/acc: 0.157 0.963
epoch 10 - train loss/acc: 0.037 0.988, valid loss/acc: 0.166 0.963
epoch 11 - train loss/acc: 0.026 0.991, valid loss/acc: 0.146 0.967
epoch 12 - train loss/acc: 0.020 0.994, valid loss/acc: 0.144 0.968
epoch 13 - train loss/acc: 0.024 0.992, valid loss/acc: 0.151 0.965
epoch 14 - train loss/acc: 0.019 0.994, valid lo

In [ ]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.141, acc = 0.967


# Extra Credit (Optional)

In [ ]:
def initialize_parameters(self, input_dim, num_hiddens, num_classes):
    """
    initializes parameters with He Initialization.

    Question (f)
    - refer to https://paperswithcode.com/method/he-initialization for He initialization 
    
    Inputs
    - input_dim
    - num_hiddens
    - num_classes
    Returns
    - params: a dictionary with the initialized parameters.
    """

    params = {}
    ##### YOUR CODE #####
    params['W1'] = np.random.randn(input_dim, num_hiddens) * (np.sqrt(2 / input_dim))
    params['b1'] = np.zeros((1, num_hiddens))
    params['W2'] = np.random.randn(num_hiddens, num_classes) * (np.sqrt(2 / num_hiddens))
    params['b2'] = np.zeros((1, num_classes))
    #####################
    return params

def forward_relu(self, X):
    """
    Defines and performs the feed forward step of a two-layer neural network.
    Specifically, the network structue is given by

        y = softmax(relu(X W1 + b1) W2 + b2)

    where X is the input matrix of shape (N, D), y is the class distribution matrix
    of shape (N, C), N is the number of examples (either the entire dataset or
    a mini-batch), D is the feature dimensionality, and C is the number of classes.

    Question (f)

    Inputs
        X: the input matrix of shape (N, D)

    Returns
        y: the output of the model
        ff_dict: a dictionary containing all the fully connected units and activations.
    """

    ff_dict = {}        
    ##### YOUR CODE #####
    ff_dict['s1'] = np.matmul(X, self.params['W1'])
    ff_dict['s2'] = ff_dict['s1'] + self.params['b1']
    ff_dict['s3'] = np.maximum(ff_dict['s2'], 0)
    ff_dict['s4'] = np.matmul(ff_dict['s3'], self.params['W2'])
    ff_dict['s5'] = ff_dict['s4'] + self.params['b2']
    y = softmax(ff_dict['s5'])
    #####################
    return y, ff_dict

def backward_relu(self, X, Y, ff_dict):
    """
    Performs backpropagation over the two-layer neural network, and returns
    a dictionary of gradients of all model parameters.

    Question (f)

    Inputs:
        - X: the input matrix of shape (B, D), where B is the number of examples
            in a mini-batch, D is the feature dimensionality.
        - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
            where B is the number of examples in a mini-batch, C is the number
            of classes.
        - ff_dict: the dictionary containing all the fully connected units and
            activations.

    Returns:
        - grads: a dictionary containing the gradients of corresponding weights
            and biases.
    """

    grads = {}
    ##### YOUR CODE #####
    dim = X.shape[1]
    dl_ds = self.forward(X)[0] - Y

    ds_1 = np.matmul(dl_ds, self.params['W2'].T) * np.greater(ff_dict['s3'],0).astype(float)
    grads['dW1'] = np.matmul(X.T, ds_1)
    grads['db1'] = (1/dim) * np.sum(ds_1, axis=0, keepdims=True)
    grads['dW2'] = np.matmul(ff_dict['s3'].T, dl_ds)
    grads['db2'] = (1/dim) * np.sum(dl_ds, axis=0, keepdims=True)
    #####################
    return grads

TwoLayerNNRelu = copy.copy(TwoLayerNN)
TwoLayerNNRelu.initialize_parameters = initialize_parameters
TwoLayerNNRelu.forward = forward_relu
TwoLayerNNRelu.backward = backward_relu

In [ ]:
### 
# Question (f)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

In [ ]:
# model instantiation
model_relu = TwoLayerNNRelu(input_dim=784, num_hiddens=64, num_classes=10)

In [ ]:
# train the model
lr, n_epochs, batch_size = 0.5, 20, 128
history = model_relu.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.186 0.944, valid loss/acc: 0.206 0.939
epoch 01 - train loss/acc: 0.120 0.964, valid loss/acc: 0.148 0.956
epoch 02 - train loss/acc: 0.085 0.976, valid loss/acc: 0.122 0.964
epoch 03 - train loss/acc: 0.068 0.980, valid loss/acc: 0.112 0.967
epoch 04 - train loss/acc: 0.065 0.980, valid loss/acc: 0.113 0.966
epoch 05 - train loss/acc: 0.058 0.983, valid loss/acc: 0.113 0.966
epoch 06 - train loss/acc: 0.048 0.986, valid loss/acc: 0.106 0.968
epoch 07 - train loss/acc: 0.048 0.985, valid loss/acc: 0.113 0.966
epoch 08 - train loss/acc: 0.037 0.989, valid loss/acc: 0.108 0.970
epoch 09 - train loss/acc: 0.028 0.993, valid loss/acc: 0.098 0.972
epoch 10 - train loss/acc: 0.044 0.986, valid loss/acc: 0.118 0.969
epoch 11 - train loss/acc: 0.025 0.993, valid loss/acc: 0.103 0.971
epoch 12 - train loss/acc: 0.020 0.995, valid loss/acc: 0.101 0.974
epoch 13 - train loss/acc: 0.020 0.995, valid loss/acc: 0.103 0.972
epoch 14 - train loss/acc: 0.017 0.996, valid lo

In [ ]:
Y_hat, _ = model_relu.forward(X_test)
test_loss = model_relu.compute_loss(Y_test, Y_hat)
test_acc = model_relu.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.088, acc = 0.977


- Discussed with: 강창훈(데이터사이언스학과)
- Reference: lab1 (for question a)